# Apply Model

This notebook gives examples of how to run a trained model on external data.

In [1]:
%matplotlib inline

import datetime

from shapely.geometry import Point, mapping, shape
import fiona

import numpy as np

import rtree

from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.externals import joblib

import matplotlib
import matplotlib.pyplot as plt

In [2]:
# BUILDING_FEATURES_FN should be a numpy file (.npy) that contains a matrix where each row represents a building and
# and each column represents a feature. There should be 24 columns, where the first 4 columns are: square footage,
# cooling degree days, heating degree days, and square footage. The remaining 20 columns should be a one-hot encoding
# principal building activity.
BUILDING_FEATURES_FN = "data/dummyFeatures.npy"

# BUILDING_LOCATIONS_FN should be a numpy file that contains a matrix with the same number of rows as BUILDING_FEATURES_FN,
# and two columns. The first column should represent the latitude of the buildings, the second column should represent the
# longitude of the buildings.
BUILDING_LOCATIONS_FN = "data/dummyFeatureLocations.npy"

# MODEL_FILE should be the path to a pre-trained model file.
MODEL_FILE = "output/trainedModels/XGBoost_trained.p"


# AGGREGATION_SHAPES_FN should be the filename of a shapefile that contains the zones with which you want to aggregate
# the individual predictions into
AGGREGATION_SHAPES_FN = "data/shapefiles/Model_Traffic_Analysis_Zones_2000.shp"
AGGREGATION_PK = "TAZ05" # Primary key to use from the AGGREGATION_SHAPES_FN shapefile

# Load Data

In [3]:
X = np.load(BUILDING_FEATURES_FN)
points = np.load(BUILDING_LOCATIONS_FN)

print X.shape
print points.shape

(5099, 24)
(5099, 2)


In [4]:
regressor = joblib.load(MODEL_FILE)
scaler = joblib.load("output/scaler.p")

# Apply Model

In [5]:
predictions = regressor.predict(scaler.transform(X))

# Perform Intersect with zone boundaries

In [6]:
index = rtree.index.Index()

pointList = []
pointEnergyList = []
for i, (lat, lon) in enumerate(points):
    geom = Point(lon, lat)
    predictedLogEnergyUse = predictions[i]
   
    pointList.append(geom)
    pointEnergyList.append(predictedLogEnergyUse)


zoneList = []
zoneIdList = []
zoneSumEnergyList = []
zonePointCount = []
f = fiona.open(AGGREGATION_SHAPES_FN)
for fid, pol in enumerate(f):
    geom = shape(pol['geometry'])
    pk = pol["properties"][AGGREGATION_PK]
    
    zoneList.append(geom)
    zoneIdList.append(pk)
    
    zoneSumEnergyList.append(0.0)
    zonePointCount.append(0.0)
    
    index.insert(fid, geom.bounds)
f.close()


for i, point in enumerate(pointList):
    for j in index.intersection(point.coords[0]):
        if zoneList[j].contains(point):
            zoneSumEnergyList[j] += (10.0**pointEnergyList[i])
            zonePointCount[j] += 1

# Write Updated TAZ boundaries

In [8]:
totalEnergy = 0.0
with fiona.open(AGGREGATION_SHAPES_FN, "r") as source:

    sink_schema = source.schema.copy()
    sink_schema['properties']['sumEnergy'] = 'float'
    sink_schema['properties']['pointCount'] = 'float'
    sink_schema['properties']['timestamp'] = 'datetime'


    with fiona.open(
        'output/ModelOutput.shp', 'w',
        crs=source.crs,
        driver=source.driver,
        schema=sink_schema,
    ) as sink:
        print "Writing source file"
        for f in source:
            
            sourceZoneID = f["properties"][AGGREGATION_PK]
            newIndex = zoneIdList.index(sourceZoneID)
            
            totalEnergy += zoneSumEnergyList[newIndex]
                
            f['properties'].update(
                sumEnergy=zoneSumEnergyList[newIndex],
                pointCount=zonePointCount[newIndex],
                timestamp=datetime.datetime.now().isoformat()
            )

            sink.write(f)
            
        print "Finished writing source file"

Writing source file
Finished writing source file
